In [1]:
import numpy as np                            #导入numpy库并简写为np
import mindquantum as mq                      #导入mindquantum库并简写为mq
from mindquantum.core import Circuit               #导入Circuit模块，用于搭建量子线路
from mindquantum.core import H, RX, RY, RZ    #导入量子门H, RX, RY, RZ

In [2]:
encoder = Circuit()                   #初始化量子线路
encoder += H.on(0)                    #H门作用在第0位量子比特
encoder += RX(f'alpha{0}').on(0)      #RX(alpha_0)门作用在第0位量子比特
encoder += RY(f'alpha{1}').on(0)      #RY(alpha_1)门作用在第0位量子比特
encoder += RZ(f'alpha{2}').on(0)      #RZ(alpha_2)门作用在第0位量子比特
encoder = encoder.no_grad()           #Encoder作为整个量子神经网络的第一层，不用对编码线路中的梯度求导数，因此加入no_grad()
encoder.summary()                     #总结Encoder
encoder

==================Circuit Summary==================
|Total number of gates  : 4.                      |
|Parameter gates        : 3.                      |
|with 3 parameters are  : alpha0, alpha1, alpha2. |
|Number qubit of circuit: 1                       |


q0: ──H────RX(alpha0)────RY(alpha1)────RZ(alpha2)──

In [3]:
alpha0, alpha1, alpha2 = 0.2, 0.3, 0.4              #alpha0, alpha1, alpha2为已知的固定值，分别赋值0.2, 0.3 和0.4
state = encoder.get_qs(pr={'alpha0': alpha0, 'alpha1': alpha1, 'alpha2': alpha2}, ket=True)
print(state)

(0.5669903122552596-0.1753906567580312j)¦0⟩
(0.800814626197614+0.08034947292077024j)¦1⟩


In [4]:
ansatz = Circuit()                           #初始化量子线路
ansatz += RX(f'theta{0}').on(0)              #RX(theta_0)门作用在第0位量子比特
ansatz += RY(f'theta{1}').on(0)              #RY(theta_1)门作用在第0位量子比特
ansatz    

q0: ──RX(theta0)────RY(theta1)──

In [5]:
theta0, theta1 = 0, 0                        #对theta0, theta1进行赋值，设为初始值0, 0
state = ansatz.get_qs(pr=dict(zip(ansatz.params_name, [theta0, theta1])), ket=True)
print(state)

1¦0⟩


In [6]:
circuit = encoder + ansatz                   #完整的量子线路由Encoder和Ansatz组成
circuit

q0: ──H────RX(alpha0)────RY(alpha1)────RZ(alpha2)────RX(theta0)────RY(theta1)──

In [7]:
from mindquantum.core import QubitOperator           #导入QubitOperator模块，用于构造泡利算符
from mindquantum.core import Hamiltonian             #导入Hamiltonian模块，用于构建哈密顿量

ham = Hamiltonian(QubitOperator('Z0', -1))           #对第0位量子比特执行泡利Z算符测量，且将系数设置为-1，构建对应的哈密顿量
print(ham)

-1 [Z0] 


In [8]:
encoder_names = encoder.params_name                   # Encoder中所有参数组成的数组，encoder.para_name系统会自动生成
ansatz_names = ansatz.params_name                     # Ansatz中所有参数组成的数组，ansatz.para_name系统会自动生成

print('encoder_names = ', encoder.params_name, '\nansatz_names =', ansatz.params_name)

encoder_names =  ['alpha0', 'alpha1', 'alpha2'] 
ansatz_names = ['theta0', 'theta1']


In [9]:
# 导入Simulator模块
from mindquantum.simulator import Simulator

# 生成一个基于projectq后端的模拟器，并设置模拟器的比特数为量子线路的比特数。
sim = Simulator('projectq', circuit.n_qubits)

# 获取模拟器基于当前量子态的量子线路演化以及期望、梯度求解算子
grad_ops = sim.get_expectation_with_grad(ham, #目标负的puli z
                                         circuit, #电路
                                         encoder_params_name=encoder_names,
                                         #告诉ops，这是经典数据
                                         ansatz_params_name=ansatz_names) #待训练的

# Encoder中的alpha0, alpha1, alpha2这三个参数组成的数组，
# 将其数据类型转换为float32，并储存在encoder_data中。
# MindQuantum支持多样本的batch训练，Encoder数组是两个维度，
# 第一个维度为样本，第二个维度为特征（即参数）
encoder_data = np.array([[alpha0, alpha1, alpha2]]).astype(np.float32)

# Ansatz中的theta0, theta1这两个参数组成的数组，将其数据类型转换为float32，
# 并储存在ansatzr_data中，Ansatz数据只有一个维度，特征（即参数）
ansatz_data = np.array([theta0, theta1]).astype(np.float32)

# 根据Encoder和Ansatz的数据，输出参数化量子线路的测量值，Encoder中的参数的导数和Ansatz中的参数的导数
measure_result, encoder_grad, ansatz_grad = grad_ops(encoder_data, ansatz_data)

print('Measurement result: ', measure_result)
print('Gradient of encoder parameters: ', encoder_grad)
print('Gradient of ansatz parameters: ', ansatz_grad)

Measurement result:  [[0.29552022+0.j]]
Gradient of encoder parameters:  [[[0.+0.j 0.+0.j 0.+0.j]]]
Gradient of ansatz parameters:  [[[-0.37202556+0.j  0.87992317+0.j]]]


In [10]:
from mindquantum.framework import MQLayer          # 导入MQLayer
import mindspore as ms                             # 导入mindspore

ms.set_seed(1)                                     #设置生成随机数的种子
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")

QuantumNet = MQLayer(grad_ops)
QuantumNet

MQLayer<
  (evolution): MQOps<1 qubit projectq VQA Operator>
  >

In [11]:
from mindspore import nn                                          #导入nn模块，nn即经典神经网络
from mindspore.nn import Adam, TrainOneStepCell                   #导入Adam模块和TrainOneStepCell模块

opti = Adam(QuantumNet.trainable_params(), learning_rate=0.5)     #需要优化的是Quantumnet中可训练的参数，学习率设为0.5   
net = TrainOneStepCell(QuantumNet, opti)

for i in range(200):
    res = net(ms.Tensor(encoder_data))
    if i % 10 == 0:
        print(i, ': ', res)

[WARNING] DEBUG(3391,7f307ec4d740,python):2021-10-30-13:17:08.671.945 [mindspore/ccsrc/debug/debugger/debugger.cc:88] Debugger] Not enabling debugger. Debugger does not support CPU.
0 :  [[0.2837115]]
10 :  [[-0.8851233]]
20 :  [[-0.97001773]]
30 :  [[-0.9929431]]
40 :  [[-0.9939507]]
50 :  [[-0.9967015]]
60 :  [[-0.99878186]]
70 :  [[-0.9995535]]
80 :  [[-0.9999011]]
90 :  [[-0.99998033]]
100 :  [[-0.9999989]]
110 :  [[-0.99999785]]
120 :  [[-0.999997]]
130 :  [[-0.9999987]]
140 :  [[-0.9999998]]
150 :  [[-1.]]
160 :  [[-0.99999994]]
170 :  [[-1.]]
180 :  [[-1.]]
190 :  [[-1.]]


In [12]:
theta0, theta1 = QuantumNet.weight.asnumpy()

print(QuantumNet.weight.asnumpy())

[ 2.2420275 -1.0756909]


In [13]:
pr = {'alpha0': alpha0, 'alpha1': alpha1, 'alpha2': alpha2, 'theta0': theta0, 'theta1': theta1}
state = circuit.get_qs(pr=pr, ket=True)

print(state)

(0.37129760050057437-0.9285139157007681j)¦0⟩
(1.4564552975271372e-05+6.455516706194153e-07j)¦1⟩


In [14]:
state = circuit.get_qs(pr=pr)
fid = np.abs(np.vdot(state, [1, 0]))**2            #保真度fidelity为向量内积的绝对值的模平方，即计算此时量子态对应的向量与|0>态对应的向量[1,0]的内积的模平方

print(fid)

0.9999999997874573
